##  Web_Scraping_MOPH version.2

I tried to learn how to do Web scraping, this is my first time.

***Reference: https://bigdata.go.th/big-data-101/data-science/data-scraping-part-3/***

In [ ]:
import requests
import bs4
from bs4 import BeautifulSoup

import pandas as pd
from bisect import bisect

In [ ]:
# Import table from version 1
df_ids = pd.read_excel("table_id.xlsx", header=[0])

In [1]:
df_all = pd.DataFrame()

In [2]:
def get_info_table(url):
    url = f"http://203.157.10.8/hcode_2020/query_detail.php?p=3&code=000001900&oldcode=00019&status=01"
    source = requests.get(url)
    soup  = BeautifulSoup(source.content, 'lxml')
    div   = soup.find('main', id='main')
    section = div.find('section', id='services')
    container = section.find('div', class_='container')
    card = container.find('div', class_='card card_warning')
    card1 = card.find('div', class_='card-body')
    form = card1.find('form', role ='form')
    roww = form.find('div', class_='row')
    rows  = roww.find_all('div')
    
    pairs = []
    for row in rows:
        tags = row.contents
        ls_header_tags, ls_header_indices = [], []
        ls_value_tags,  ls_value_indices  = [], []
        headers_paired = []
        
        for i, tag in enumerate(tags):
            if is_header(tag):
                ls_header_tags += [tag]
                ls_header_indices+= [i]
                headers_paired += [False]
            elif is_value(tag):
                ls_value_tags  += [tag]
                ls_value_indices += [i]
        
        for i in range(0, len(ls_value_tags)):
            loc_hd = bisect(ls_header_indices, ls_value_indices[i])
            if loc_hd==0: # when value comes before any header
                header = pairs[-1][0]+"_2"
            else:
                header = ls_header_tags[loc_hd-1].text\
                        .replace("\xa0", "").replace("\n", " ")\
                        .replace("\t", " ").strip()
                headers_paired[loc_hd-1] = True
            
            value = ls_value_tags[i].text\
                        .replace("\xa0", "").replace("\n", " ")\
                        .replace("\t", " ").strip()
            
            pairs += [[header, value]]
        
        for i in range(len(ls_header_tags)):
            if not headers_paired[i]:
                header = ls_header_tags[i].text\
                            .replace("\xa0", "")\
                            .replace("\n", " ")\
                            .replace("\t", " ")\
                            .strip()
                pairs += [[header, ""]]
    
    df = pd.DataFrame(pairs)
    df.set_index([0], inplace=True)
    df.replace({"":np.nan, "-":np.nan}, inplace=True)
    df.dropna(how='any', axis='index', inplace=True)
    df.drop_duplicates(inplace=True)
    
    return df.T

In [ ]:
for i in range(1, 5):
    code    = str(df_ids.iloc[i, 1]).replace("\xa0", "")
    oldcode = str(df_ids.iloc[i, 2]).replace("\xa0", "")
    url = f"http://203.157.10.8/hcode_2020/query_detail.php?p=3&code={code}&oldcode={oldcode}&status=01"
    try:
        df     = get_info_table(url)
        df_all = pd.concat([df_all, df], axis=0, sort=False,\
                               ignore_index=True, join="outer")
        print(f"Done scraping {i}")
        df_all.to_pickle("temp.pkl")
    except Exception as e:
        print(f"Cannot scrape {i}")
        print(str(e))

#df_all.to_excel('healthcare_providers.xlsx', index=False)

In [ ]:
# 11:40 14/10/2021
# inprogress to be continue